In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ultimate-spotify-tracks-db/SpotifyFeatures.csv


In [2]:
df = pd.read_csv('/kaggle/input/ultimate-spotify-tracks-db/SpotifyFeatures.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232725 entries, 0 to 232724
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   genre             232725 non-null  object 
 1   artist_name       232725 non-null  object 
 2   track_name        232725 non-null  object 
 3   track_id          232725 non-null  object 
 4   popularity        232725 non-null  int64  
 5   acousticness      232725 non-null  float64
 6   danceability      232725 non-null  float64
 7   duration_ms       232725 non-null  int64  
 8   energy            232725 non-null  float64
 9   instrumentalness  232725 non-null  float64
 10  key               232725 non-null  object 
 11  liveness          232725 non-null  float64
 12  loudness          232725 non-null  float64
 13  mode              232725 non-null  object 
 14  speechiness       232725 non-null  float64
 15  tempo             232725 non-null  float64
 16  time_signature    23

In [3]:
df.isnull().sum()

genre               0
artist_name         0
track_name          0
track_id            0
popularity          0
acousticness        0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
key                 0
liveness            0
loudness            0
mode                0
speechiness         0
tempo               0
time_signature      0
valence             0
dtype: int64

In [4]:
y = df['popularity']
X = df.drop(columns = ['popularity'], axis = 1)

In [5]:
from sklearn.preprocessing import LabelEncoder

for col in X.columns:
    if X[col].dtype == 'object':
        X[col] = LabelEncoder().fit_transform(X[col])

X.head()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,genre,artist_name,track_name,track_id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,15,5181,18348,4210,0.611,0.389,99373,0.910,0.000,4,0.3460,-1.828,0,0.0525,166.969,3,0.814
1,15,8254,92963,4316,0.246,0.590,137373,0.737,0.000,9,0.1510,-5.559,1,0.0868,174.003,3,0.816
2,15,6451,32194,4704,0.952,0.663,170267,0.131,0.000,3,0.1030,-13.879,1,0.0362,99.488,4,0.368
3,15,5181,30998,6102,0.703,0.240,152427,0.326,0.000,4,0.0985,-12.178,0,0.0395,171.758,3,0.227
4,15,4074,90793,6973,0.950,0.331,82625,0.225,0.123,8,0.2020,-21.150,0,0.0456,140.576,3,0.390


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 21)
y_test = pd.array(y_test, dtype = np.dtype("int"))

In [7]:
#очень наврятли, что зависимость линейна, но посмотрим, какой скор даст линейная регрессия
from sklearn.linear_model import LinearRegression


linear_model = LinearRegression()

linear_model.fit(X_train, y_train)

predict_y = linear_model.predict(X_test)

linear_MAE = 0

for i in range(0, len(predict_y)):
    linear_MAE = linear_MAE + abs(y_test[i] - predict_y[i])
    
linearMAE = linear_MAE / len(predict_y)

In [8]:
#попробуем дерево регрессии
from sklearn.tree import DecisionTreeRegressor

tree_regr = DecisionTreeRegressor(max_depth = 12)

tree_regr.fit(X_train, y_train)

tree_regr_predict_y = tree_regr.predict(X_test)

tree_regr_MAE = 0

for i in range(0, len(tree_regr_predict_y)):
    tree_regr_MAE = tree_regr_MAE + abs(y_test[i] - tree_regr_predict_y[i])
    
tree_regr_MAE = tree_regr_MAE / len(predict_y)

In [9]:
#попробуем лес деревьев
from sklearn.ensemble import RandomForestRegressor

forest_regr = RandomForestRegressor(n_estimators = 400, max_depth = 3, random_state = 21)

forest_regr.fit(X_train, y_train)

forest_regr_predict_y = forest_regr.predict(X_test)

forest_regr_MAE = 0

for i in range(0, len(forest_regr_predict_y)):
    forest_regr_MAE = forest_regr_MAE + abs(y_test[i] - forest_regr_predict_y[i])
     
forest_regr_MAE = forest_regr_MAE / len(predict_y)




In [10]:
#Попробуем GBM
from sklearn.ensemble import GradientBoostingRegressor

GBM_reg = GradientBoostingRegressor(random_state = 21, n_estimators = 100, max_depth = 3, learning_rate = 0.1)

GBM_reg.fit(X_train, y_train)

GBM_regr_predict_y = GBM_reg.predict(X_test)

GBM_regr_MAE = 0

for i in range(0, len(GBM_regr_predict_y)):
    GBM_regr_MAE = GBM_regr_MAE + abs(y_test[i] - GBM_regr_predict_y[i])
    
GBM_regr_MAE = GBM_regr_MAE / len(predict_y)

In [11]:
#Наконец, попробуем нейронные сети прямого распространения
from keras.models import Sequential
from keras import layers

neural_model = Sequential()

neural_model.add(layers.Dense(8, input_dim = 17, activation = 'relu'))
neural_model.add(layers.Dense(6, activation = 'relu'))
neural_model.add(layers.Dense(1, activation = 'linear'))

neural_model.compile(optimizer = "Adam", loss="mae", metrics=["mae"])

neural_model.fit(X_train, y_train, epochs = 6, batch_size = 5)

neural_predict_y = neural_model.predict(X_test)

neural_MAE = 0

for i in range(0, len(neural_predict_y)):
    neural_MAE = neural_MAE + abs(y_test[i] - neural_predict_y[i])
    
neural_MAE = neural_MAE / len(predict_y)

#перепробовав множество возможных внешних параметров, более удачного результата получить не вышло

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Epoch 1/6
32582/32582 [==============================] - 47s 1ms/step - loss: 185.0571 - mae: 185.0571
Epoch 2/6
32582/32582 [==============================] - 46s 1ms/step - loss: 16.5635 - mae: 16.5635
Epoch 3/6
32582/32582 [==============================] - 46s 1ms/step - loss: 15.8220 - mae: 15.8220
Epoch 4/6
32582/32582 [==============================] - 46s 1ms/step - loss: 14.8063 - mae: 14.8063
Epoch 5/6
32582/32582 [==============================] - 46s 1ms/step - loss: 14.8067 - mae: 14.8067
Epoch 6/6
2182/2182 [==============================] - 2s 1ms/step


In [12]:
#Итого
print('linearMAE:', linearMAE)
print('tree_regr_MAE:', tree_regr_MAE)
print('forest_regr_MAE:', forest_regr_MAE)
print('GBM_regr_MAE:', GBM_regr_MAE)
print('neural_MAE:', neural_MAE)

linearMAE: 12.654820059916071
tree_regr_MAE: 7.148530607604506
forest_regr_MAE: 11.072166201472854
GBM_regr_MAE: 7.642583376819661
neural_MAE: [14.75543019]
